# Game Manager

## Setup

In [5]:
%pip install -r requirements.txt
%pip install ipywidgets
%pip install ipyfilechooser

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Management GUI

In [1]:
import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output

from dataclasses import dataclass, field, asdict
import datetime

from src.ScraperRepository import ScraperRepository
from src.GameScraper import GameScraper
from src.Game import Game
from src.GameList import GameList

from src.scrapers.AllTheFallenGameScraper import AllTheFallenGameScraper
# from src.scrapers.DikgamesGameScraper import DikgamesGameScraper
from src.scrapers.F95zoneGameScraper import F95zoneGameScraper
from src.scrapers.FapNationGameScraper import FapNationGameScraper
from src.scrapers.LewdCornerGamerScraper import LewdCornerGamerScraper


repository = ScraperRepository()
repository.add(AllTheFallenGameScraper)
# repository.add(DikgamesGameScraper)
repository.add(F95zoneGameScraper)
repository.add(FapNationGameScraper)
repository.add(LewdCornerGamerScraper)

gamelist = GameList(repository=repository)
gamelist.load()

url_input = widgets.Text(description="Url:")
add_from_url_button = widgets.Button(description="Add game from Url")
def add_from_url(b):
    url = url_input.value
    if url:
        gamelist.add_game_from_url(url)
    print("DONE")
add_from_url_button.on_click(add_from_url)

check_url_button = widgets.Button(description="Check game from Url")
def check_url(b):
    url = url_input.value
    if url:
        game = gamelist.get_by_url(url)
        if not url:
            print(f"game with {url} not found")
        else:
            try:
                game.check_for_updates(repository=gamelist.repository) 
            except Exception as e:
                print(f"Error updating '{game.title}': {e}")
    print("DONE")
check_url_button.on_click(check_url)

force_update_url_button = widgets.Button(description="Force update on game from Url")
def force_update_from_url(b):
    url = url_input.value
    if url:
        game = gamelist.get_by_url(url)
        if not url:
            print(f"game with {url} not found")
        else:
            try:
                game.update(repository=gamelist.repository) 
            except Exception as e:
                print(f"Error updating '{game.title}': {e}")
    print("DONE")
force_update_url_button.on_click(force_update_from_url)

save_button = widgets.Button(description="Save gamelist")
def save_changes(b):
    gamelist.save()
    gamelist.create_index()
save_button.on_click(save_changes)

filename_input = FileChooser(
    './',  # Default directory
    filename='urls.txt',  # Optional: Pre-fill filename
    select_default=False,  # Start without pre-selected file
    title='Select a fext file with URLs to add',  # Title displayed above the widget
)
add_from_file_button = widgets.Button(description="Start import")
def import_from_file(b):
    selected_file = filename_input.selected
    if selected_file:
        try:
            with open(selected_file, 'r', encoding='utf-8') as file:
                for line in file:
                    clean_line = line.strip()
                    # Ignore empty lines and comment lines
                    if clean_line and not clean_line.startswith("#"):
                        gamelist.add_game_from_url(clean_line)
        except Exception as e:
            print(f"Error reading file: {e}")
    print("DONE")
add_from_file_button.on_click(import_from_file)

force_all_updates_button = widgets.Button(description="Force update all")
def force_all_updates(b):
    for game in gamelist.games:
        try:
            game.update(repository=gamelist.repository) 
        except Exception as e:
            print(f"Error updating '{game.title}': {e}")
    print("DONE")
force_all_updates_button.on_click(force_all_updates)

check_all_for_updates_button = widgets.Button(description="Check all for updates")
def check_all_for_updates(b):
    print(f"Checking for updates, {datetime.datetime.now()}")
    updates = gamelist.check_for_updates(immediate_update=True)
    print(f"{len(updates)} updates found, {datetime.datetime.now()}")
    print("DONE")
check_all_for_updates_button.on_click(check_all_for_updates)

add_from_url_ui = widgets.HBox([url_input, add_from_url_button, check_url_button, force_update_url_button])
add_from_url_file = widgets.HBox([filename_input, add_from_file_button])
updates_ui = widgets.HBox([force_all_updates_button, check_all_for_updates_button])
display(add_from_url_ui, add_from_url_file, updates_ui, save_button)

Loaded 8 games


Button(description='Save gamelist', style=ButtonStyle())